In [12]:
!pip install fiftyone
!pip install fiftyone-db==0.4.3
import torch


In [13]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
%cd /content
!git clone https://<token>@github.com/aslakdjupskas/STF.git
!git pull
!pip install compressai
!pip install pybind11


Cloning into 'STF'...
remote: Enumerating objects: 343, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 343 (delta 65), reused 75 (delta 41), pack-reused 232
Receiving objects: 100% (343/343), 28.26 MiB | 21.03 MiB/s, done.
Resolving deltas: 100% (141/141), done.
Already up to date.


In [16]:
# Loading pretrained weights
%mkdir -p compressai/pretrained && gdown -O compressai/pretrained/ 1OFzZoEaofNgsimBuOPHtgOJiGsR_RS-M
!pip install -r requirements.txt
!pip install -e .
!pip install -e '.[dev]'


Downloading...
From: https://drive.google.com/uc?id=1OFzZoEaofNgsimBuOPHtgOJiGsR_RS-M
To: /content/STF/STF/compressai/pretrained/stf_0035_best.pth.tar
100% 1.20G/1.20G [00:18<00:00, 64.3MB/s]
  Using cached alabaster-0.7.12-py2.py3-none-any.whl (14 kB)
  Using cached appnope-0.1.2-py2.py3-none-any.whl (4.3 kB)
  Using cached argon2_cffi-20.1.0-cp35-abi3-manylinux1_x86_64.whl (97 kB)
  Using cached astroid-2.5.6-py3-none-any.whl (219 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached Babel-2.9.0-py2.py3-none-any.whl (8.8 MB)
  Using cached beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
  Using cached black-21.5b2-py3-none-any.whl (138 kB)
  Using cached bleach-3.3.0-py2.py3-none-any.whl (283 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached cffi-1.14.5.tar.gz (475 kB)
  Preparing metadata (setup.py) ... done
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Us

In [17]:

!git checkout devMorten
!git pull


M	compressai/version.py
Branch 'devMorten' set up to track remote branch 'devMorten' from 'origin'.
Switched to a new branch 'devMorten'
Already up to date.


In [18]:
# !pip install torch

from optimized_compressions import (compress_by_y_optimization_through_decoder,
                                    decompress_by_image_optimization_through_encoder,
                                    y_bar_optimize_from_imagedecoder)

from our_utils import (pull_openimages,
                       prepare_data_loader,
                       load_pretrained_model,
                       plot_reconstruction)

import matplotlib.pyplot as plt
#%matplotlib notebook
#%matplotlib widget
%matplotlib inline

In [ ]:
# Some defenitions
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset_dir = "openimages"
channels = 3
test_batch_size = 2
patch_size = (256, 256)

# Prepare data and load model
pull_openimages(traning_size=test_batch_size+1, test_size=test_batch_size, dataset_dir=dataset_dir)
test_dataloader = prepare_data_loader(patch_size=patch_size, test_batch_size=test_batch_size, device=device, dataset_dir=dataset_dir)
model = load_pretrained_model(path="compressai/pretrained/stf_0035_best.pth.tar", device=device, freeze=True)

our_batch = next(iter(test_dataloader))
our_batch = our_batch.to(device)

# Generate a normal reconstruction
standard_compression = model.compress(our_batch)
normal_reconstruction = model.decompress(*standard_compression.values())['x_hat']



INFO:fiftyone.zoo.datasets:Downloading split 'train' to 'openimages/train' if necessary


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv' to 'openimages/train/metadata/image_ids.csv'


 100% |██████|    4.8Gb/4.8Gb [4.9s elapsed, 0s remaining, 814.8Mb/s]       


INFO:eta.core.utils: 100% |██████|    4.8Gb/4.8Gb [4.9s elapsed, 0s remaining, 814.8Mb/s]       


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv' to 'openimages/train/metadata/classes.csv'


...

Dataset info written to 'openimages/info.json'


INFO:fiftyone.zoo.datasets:Dataset info written to 'openimages/info.json'


Loading existing dataset 'open-images-v6-test-2'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'open-images-v6-test-2'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Downloading split 'train' to 'openimages/train' if necessary


In [ ]:
compression_by_y = compress_by_y_optimization_through_decoder(model, our_batch, normal_reconstruction, iterations=3)
reconstructed_image_y_compression = model.decompress(*compression_by_y.values())['x_hat']

# Plot the results
plot_reconstruction(our_batch, reconstructed_image_y_compression, normal_reconstruction)
plt.show(block=True)

In [ ]:
reconstruction_by_y_bar_standar_compression = decompress_by_image_optimization_through_encoder(model, standard_compression, normal_reconstruction, iterations=3)
plot_reconstruction(our_batch, reconstruction_by_y_bar_standar_compression, normal_reconstruction)

In [ ]:
reconstructed_by_bar_y_compression = decompress_by_image_optimization_through_encoder(model, compression_by_y, normal_reconstruction, iterations=3)
plot_reconstruction(our_batch, reconstructed_by_bar_y_compression, normal_reconstruction)